# Problem statement 2: Data Augmentation and Synthesis

#Importing all dependencies and loading dataset, Sentiment score analysis and EDA on missing and null values

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as sk
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
# Loading Review Dataset
review_data=pd.read_csv("/content/drive/MyDrive/Trimester_3/Capstone_project/Datasets/reviews.csv")
review_data

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,958,5977,2009-07-23,15695,Edmund C,"Our experience was, without a doubt, a five st..."
1,958,6660,2009-08-03,26145,Simon,Returning to San Francisco is a rejuvenating t...
2,958,11519,2009-09-27,25839,Denis,We were very pleased with the accommodations a...
3,958,16282,2009-11-05,33750,Anna,We highly recommend this accomodation and agre...
4,958,26008,2010-02-13,15416,V Jillian,Holly's place was great. It was exactly what I...
...,...,...,...,...,...,...
392954,1219622445222452442,1230654319092135704,2024-08-24,51979490,Solène,D
392955,1219622445222452442,1232036398314385400,2024-08-26,18919150,Kristopher,Great stay. Key vob stopped working but host a...
392956,1219785829994931401,1237167935539710683,2024-09-02,4338272,Anastasia,Great choice for staying! The apartment is ama...
392957,1233574822709070023,1237138800668239871,2024-09-02,541938241,Dave,Wonderful


In [4]:
review_data.shape

(392959, 6)

In [5]:
review_data.value_counts()

listing_id           id                   date        reviewer_id  reviewer_name  comments                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
958                  5977                 2009-07-23  15695        Edmund C       Our experience was, without a doubt, a five star experience. Holly and her husband, David, were the consummate hosts; friendly and accomodating while still honoring our privacy. The apartment was a charming layout with a full view and access to the home's garden The location is perfect for full engagement with the city; close to mass transit with walking proximity to the Haight, the Mission, the Castro and Golden Gate Park. I can't wait for our next visit.\r<br/>\r<br/>Ted and Karen Wingerd                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         1
30069424             624869237            2020-05-15  81954409     Janiqua        I’ll start with the good. This is a really great place as far as getting an apt in a nice area, with a garage parking spot that you can actually fit a car in! It is also full of all the amenities you could need. The bathrooms are GORGEOUS! You also share the W/D with the neighbor, but still very convenient. Edwin is very responsive and always answered in a timely manner. On the flip side, there were more spiders than I cared to see. The walls were pretty scuffed when we arrived as well in nearly every room, so we had to magic erase those ourselves unfortunately. But the worst part of this stay was the upstairs neighbor. Beware! He sings nearly everyday, at the top of his lungs, with complete disregard for the people below him. It wouldn’t be so bad if you couldn’t hear him so clearly, but unfortunately you can and that’s even with ear plugs in. In addition, it’s just a lot of other miscellaneous noise that became the norm and got old quick.  Keep in mind I stayed here for 3 mos, so if you’re in and out you might not experience all of these issues. Just wanted to give a heads up I wish I had.    1
30091565             353128547            2018-11-26  12429185     Daylon         Good spot for a quick weekend stay in a central part of the city!                                                                                                                                                                                                                                                                              

In [6]:
review_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 392959 entries, 0 to 392958
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   listing_id     392959 non-null  int64 
 1   id             392959 non-null  int64 
 2   date           392959 non-null  object
 3   reviewer_id    392959 non-null  int64 
 4   reviewer_name  392956 non-null  object
 5   comments       392889 non-null  object
dtypes: int64(3), object(3)
memory usage: 18.0+ MB


In [7]:
review_data.columns

Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'], dtype='object')

In [8]:
review_data.describe()

,listing_id,id,reviewer_id
count,3.929590e+05,3.929590e+05,3.929590e+05
mean,8.225927e+16,4.223165e+17,1.251442e+08
std,2.459846e+17,4.557533e+17,1.407015e+08
min,9.580000e+02,1.981000e+03,1.000000e+00
25%,4.359078e+06,2.791330e+08,1.992458e+07
50%,1.742188e+07,7.701426e+08,6.663371e+07
75%,3.680482e+07,8.646046e+17,1.818866e+08
max,1.234137e+18,1.237913e+18,5.984690e+08


#Importing NLTK for sentiment score analysis.

In [9]:
# Installing nltk
!pip install nltk

In [10]:
# Importing all the dependencies for nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('vader_lexicon')
lemma = WordNetLemmatizer()
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [11]:
# Using vader, we are importing sentiment analyzer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# Load the sentiment analyzer
sent = SentimentIntensityAnalyzer()
review_final = pd.read_csv('/content/drive/MyDrive/Trimester_3/Capstone_project/Datasets/reviews.csv', usecols=['comments'])
review_final['comments'] = review_final['comments'].astype(str)

# Calculate polarity scores
polarity = [round(sent.polarity_scores(i)['compound'], 2) for i in review_final['comments']]
review_final['sentiment_score'] = polarity
review_final.head()

,comments,sentiment_score
0,"Our experience was, without a doubt, a five st...",0.96
1,Returning to San Francisco is a rejuvenating t...,0.98
2,We were very pleased with the accommodations a...,0.76
3,We highly recommend this accomodation and agre...,0.98
4,Holly's place was great. It was exactly what I...,0.96


In [12]:
review_total = pd.merge(review_data, review_final, on='comments', how='inner')
review_total

,listing_id,id,date,reviewer_id,reviewer_name,comments,sentiment_score
0,958,5977,2009-07-23,15695,Edmund C,"Our experience was, without a doubt, a five st...",0.96
1,958,6660,2009-08-03,26145,Simon,Returning to San Francisco is a rejuvenating t...,0.98
2,958,11519,2009-09-27,25839,Denis,We were very pleased with the accommodations a...,0.76
3,958,16282,2009-11-05,33750,Anna,We highly recommend this accomodation and agre...,0.98
4,958,26008,2010-02-13,15416,V Jillian,Holly's place was great. It was exactly what I...,0.96
...,...,...,...,...,...,...,...
1219586,1233574822709070023,1237138800668239871,2024-09-02,541938241,Dave,Wonderful,0.57
1219587,1233574822709070023,1237138800668239871,2024-09-02,541938241,Dave,Wonderful,0.57
1219588,1233574822709070023,1237138800668239871,2024-09-02,541938241,Dave,Wonderful,0.57
1219589,1233574822709070023,1237138800668239871,2024-09-02,541938241,Dave,Wonderful,0.57


In [13]:
#def preprocess_reviews(review_final):
 # reviews = review_final
  #reviews = review_final.str.replace(r'[^a-z\s]', '', regex=True)
  #return reviews

In [14]:
# review_final['comments'] = preprocess_reviews(review_final['comments'])
# review_final.head(100)

In [15]:
review_total.to_csv('review_total.csv', index=False)

In [16]:
review_total = pd.read_csv('/content/review_total.csv')
review_total

<ipython-input-16-33898a20d8ca>:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  review_total = pd.read_csv('/content/review_total.csv')


,listing_id,id,date,reviewer_id,reviewer_name,comments,sentiment_score
0,958,5.977000e+03,2009-07-23,15695.0,Edmund C,"Our experience was, without a doubt, a five st...",0.96
1,958,6.660000e+03,2009-08-03,26145.0,Simon,Returning to San Francisco is a rejuvenating t...,0.98
2,958,1.151900e+04,2009-09-27,25839.0,Denis,We were very pleased with the accommodations a...,0.76
3,958,1.628200e+04,2009-11-05,33750.0,Anna,We highly recommend this accomodation and agre...,0.98
4,958,2.600800e+04,2010-02-13,15416.0,V Jillian,Holly's place was great. It was exactly what I...,0.96
...,...,...,...,...,...,...,...
1223497,1233574822709070023,1.237139e+18,2024-09-02,541938241.0,Dave,Wonderful,0.57
1223498,1233574822709070023,1.237139e+18,2024-09-02,541938241.0,Dave,Wonderful,0.57
1223499,1233574822709070023,1.237139e+18,2024-09-02,541938241.0,Dave,Wonderful,0.57
1223500,1233574822709070023,1.237139e+18,2024-09-02,541938241.0,Dave,Wonderful,0.57


In [17]:
review_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1223502 entries, 0 to 1223501
Data columns (total 7 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   listing_id       1223502 non-null  object 
 1   id               1221134 non-null  float64
 2   date             1219591 non-null  object 
 3   reviewer_id      1219591 non-null  float64
 4   reviewer_name    1219588 non-null  object 
 5   comments         1219573 non-null  object 
 6   sentiment_score  1218048 non-null  float64
dtypes: float64(3), object(4)
memory usage: 65.3+ MB


In [44]:
count_duplicates = review_total.duplicated().sum()
print(f"Number of duplicate rows: {count_duplicates}")

Number of duplicate rows: 0


In [19]:
review_total.value_counts()

listing_id           id            date        reviewer_id  reviewer_name  comments                                                                                                                                                                                                                                                                                                                                                                                                                            sentiment_score
36029332             5.630902e+08  2019-11-11  305392993.0  Christopher    Great place!                                                                                                                                                                                                                                                                                                                                                                                                                        0.66               366
1071874400412409288  1.090801e+18  2024-02-13  5471682.0    Nick           Great place!                                                                                                                                                                                                                                                                                                                                                                                                                        0.66               366
13293285             3.338120e+08  2018-10-07  5221795.0    Christina      Great place!                                                                                                                                                                                                                                                                                                                                                                                                                        0.66               366
21231615             3.356084e+08  2018-10-12  185080062.0  Sam            Great place!                                                                                                                                                                                                                                                                                                                                                                                                                        0.66               366
49634091             5.783796e+17  2022-03-08  435495556.0  Rebekah        Great place!                                                                                                                                                                                                                                                                                                                                                                                                                        0.66               366
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 ... 
51050512             7.493044e+17  2022-10-30  51298116.0   Victoria       It was a great location felt super safe and central to a lot of places.                                                                                                                                                                                                                                           

In [20]:
review_total.columns

Index(['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments',
       'sentiment_score'],
      dtype='object')

In [21]:
review_total.describe()

,id,reviewer_id,sentiment_score
count,1.221134e+06,1.219591e+06,1.218048e+06
mean,4.902117e+17,1.707257e+08,6.182413e-01
std,4.557731e+17,1.561722e+08,2.819883e-01
min,-9.800000e-01,1.000000e+00,-1.000000e+00
25%,4.654292e+08,3.825521e+07,6.200000e-01
50%,5.435345e+17,1.220641e+08,6.600000e-01
75%,9.160395e+17,2.643667e+08,7.600000e-01
max,1.237913e+18,5.984690e+08,1.000000e+00


In [22]:
review_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1223502 entries, 0 to 1223501
Data columns (total 7 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   listing_id       1223502 non-null  object 
 1   id               1221134 non-null  float64
 2   date             1219591 non-null  object 
 3   reviewer_id      1219591 non-null  float64
 4   reviewer_name    1219588 non-null  object 
 5   comments         1219573 non-null  object 
 6   sentiment_score  1218048 non-null  float64
dtypes: float64(3), object(4)
memory usage: 65.3+ MB


In [23]:
review_total.isnull().sum()

,0
listing_id,0
id,2368
date,3911
reviewer_id,3911
reviewer_name,3914
comments,3929
sentiment_score,5454


In [24]:
review_total.dropna(inplace=True)

In [25]:
review_total.isnull().sum()
review_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1218045 entries, 0 to 1223501
Data columns (total 7 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   listing_id       1218045 non-null  object 
 1   id               1218045 non-null  float64
 2   date             1218045 non-null  object 
 3   reviewer_id      1218045 non-null  float64
 4   reviewer_name    1218045 non-null  object 
 5   comments         1218045 non-null  object 
 6   sentiment_score  1218045 non-null  float64
dtypes: float64(3), object(4)
memory usage: 74.3+ MB


In [26]:
count_duplicates = review_total.duplicated().sum()
print(f"Number of duplicate rows: {count_duplicates}")

Number of duplicate rows: 826702


In [27]:
review_total.drop_duplicates(inplace=True)

In [28]:
review_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391343 entries, 0 to 1223501
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   listing_id       391343 non-null  object 
 1   id               391343 non-null  float64
 2   date             391343 non-null  object 
 3   reviewer_id      391343 non-null  float64
 4   reviewer_name    391343 non-null  object 
 5   comments         391343 non-null  object 
 6   sentiment_score  391343 non-null  float64
dtypes: float64(3), object(4)
memory usage: 23.9+ MB


In [29]:
review_total.value_counts()

listing_id  id            date        reviewer_id  reviewer_name  comments                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              sentiment_score
18794629    3.658128e+08  2019-01-01  188313461.0  Jessie         The Red Victorian is taken care of by very friendly and responsive hosts.                                                                                                                                                                                                                                                                                                                                                                                                                                             0.86               1
17965243    1.168235e+18  2024-05-30  32968412.0   Gia            Gene's space was a welcome respite. We had everything we needed to be more than comfortable. So much natural light and thoughtful touches everywhere, including a heater in the bathroom which was so nice on a chilly SF day (as well as great water pressure)! Easy walks to the beach, restaurants/market, and public transportation. We enjoyed both going out  and staying in. Will stay here again!                                                                                                             0.98               1
            1.200909e+18  2024-07-14  17357907.0   Renate         Gene's apartment is  very beautiful, and exactly as described. We felt at home right away, and Gene was very helpful when we wanted to drop off our luggage even before checkin time (Thank you again!).<br/>It was close to perfect, nothing comes to mind that could have been better. Note that this is a residential neighborhood, so it is a little away from SF city life, but on a sunny day the Outer Sunset is probably as beautiful as a residential neighborhood can be!                                   0.95               1
            1.199502e+18  2024-07-12  14519205.0   Serena Wilkie  This was an easy, no fuss, comfortable place to stay.                                                                                                                                                                                                                                                                                                                                                                                                                                                                 0.61               1
            1.197991e+18  2024-07-10  575965732.0  Dirk           The place is very spacious and clean. The kitchen is well equipped with a stove, coffee maker, oven, microwave, plates and utensils. The bathroom is spacious and modern with a shower. The bedrooms are well sized with closets to store belongings, and desks in both room. The whole house is excellent and it fit our vacation needs perfectly. A washer and dryer are available for use! We enjoyed our stay! Highly recommended!                                                                                0.97               1
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [30]:
object_cols = review_total.select_dtypes(include=['object']).columns
print(object_cols)

Index(['listing_id', 'date', 'reviewer_name', 'comments'], dtype='object')


In [31]:
from sklearn.preprocessing import LabelEncoder

# Define the columns to be label encoded
object_cols_to_encode = review_total[['listing_id', 'date', 'reviewer_name', 'comments']]

# Iterate through the specified columns and apply Label Encoding
for column in object_cols_to_encode:
    # Convert the column to string type before applying Label Encoding
    review_total[column] = review_total[column].astype(str)
    le = LabelEncoder()
    review_total[column] = le.fit_transform(review_total[column])

In [32]:
object_cols_to_encode = pd.DataFrame(object_cols_to_encode)

In [33]:
object_cols_to_encode.head()

,listing_id,date,reviewer_name,comments
0,958,2009-07-23,Edmund C,"Our experience was, without a doubt, a five st..."
1,958,2009-08-03,Simon,Returning to San Francisco is a rejuvenating t...
2,958,2009-09-27,Denis,We were very pleased with the accommodations a...
3,958,2009-11-05,Anna,We highly recommend this accomodation and agre...
4,958,2010-02-13,V Jillian,Holly's place was great. It was exactly what I...


In [34]:
import numpy as np

# Assuming 'id', 'reviewer_id', and 'sentiment_score' are columns in your DataFrame 'review_total'
# and you want to convert these columns to integer type:

# Select the columns you want to convert
columns_to_convert = review_total[['id', 'reviewer_id', 'sentiment_score']]

# Convert selected columns to numeric (int if possible, otherwise float)
for column in columns_to_convert:
    try:
        review_total[column] = pd.to_numeric(review_total[column], errors='coerce').astype(int)
    except ValueError:
        print(f"Column '{column}' could not be converted to int. Converting to float instead.")
        review_total[column] = pd.to_numeric(review_total[column], errors='coerce').astype(float)

In [35]:
columns_to_convert.head()

,id,reviewer_id,sentiment_score
0,5977.0,15695.0,0.96
1,6660.0,26145.0,0.98
2,11519.0,25839.0,0.76
3,16282.0,33750.0,0.98
4,26008.0,15416.0,0.96


In [36]:
review_total.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,sentiment_score
0,5883,5977,4,15695,11068,218351,0
1,5883,6660,5,26145,39116,237567,0
2,5883,11519,14,25839,9775,355658,0
3,5883,16282,18,33750,2626,345807,0
4,5883,26008,30,15416,43294,131685,0


In [37]:
review_total['sentiment_score'].unique()

array([ 0,  1, -1])

# Using CTGAN for Data synthetic data generation

In [39]:
!pip install sdv

  Using cached sdv-1.17.1-py3-none-any.whl.metadata (13 kB)
  Using cached boto3-1.35.63-py3-none-any.whl.metadata (6.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.3/152.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 3.5 MB/s eta 0:00:00


In [40]:
import pandas as pd

from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer

review_sdata=pd.read_csv('/content/review_total.csv')
review_sdata.head()


<ipython-input-40-99df904ca869>:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  review_sdata=pd.read_csv('/content/review_total.csv')


,listing_id,id,date,reviewer_id,reviewer_name,comments,sentiment_score
0,958,5977.0,2009-07-23,15695.0,Edmund C,"Our experience was, without a doubt, a five st...",0.96
1,958,6660.0,2009-08-03,26145.0,Simon,Returning to San Francisco is a rejuvenating t...,0.98
2,958,11519.0,2009-09-27,25839.0,Denis,We were very pleased with the accommodations a...,0.76
3,958,16282.0,2009-11-05,33750.0,Anna,We highly recommend this accomodation and agre...,0.98
4,958,26008.0,2010-02-13,15416.0,V Jillian,Holly's place was great. It was exactly what I...,0.96


In [41]:
review_sdata.shape

(1223502, 7)

In [50]:
count_duplicates = review_sdata.duplicated().sum()
print(f"Number of duplicate rows: {count_duplicates}")

Number of duplicate rows: 826702


In [53]:
review_sdata.drop_duplicates(inplace=True)

In [54]:
review_sdata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391343 entries, 0 to 1223501
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   listing_id       391343 non-null  object 
 1   id               391343 non-null  float64
 2   date             391343 non-null  object 
 3   reviewer_id      391343 non-null  float64
 4   reviewer_name    391343 non-null  object 
 5   comments         391343 non-null  object 
 6   sentiment_score  391343 non-null  float64
dtypes: float64(3), object(4)
memory usage: 23.9+ MB


In [55]:
count_duplicates = review_sdata.duplicated().sum()
print(f"Number of duplicate rows: {count_duplicates}")

Number of duplicate rows: 0


In [64]:
from sklearn.preprocessing import LabelEncoder

# Define the columns to be label encoded
object_cols_to_encode = review_sdata[['listing_id', 'date', 'reviewer_name', 'comments']]

# Iterate through the specified columns and apply Label Encoding
for column in object_cols_to_encode:
    # Convert the column to string type before applying Label Encoding
    review_sdata[column] = review_sdata[column].astype(str)
    le = LabelEncoder()
    review_sdata[column] = le.fit_transform(review_sdata[column])

In [65]:
object_cols_to_encode.head()

,listing_id,date,reviewer_name,comments
0,958,2009-07-23,Edmund C,"Our experience was, without a doubt, a five st..."
1,958,2009-08-03,Simon,Returning to San Francisco is a rejuvenating t...
2,958,2009-09-27,Denis,We were very pleased with the accommodations a...
3,958,2009-11-05,Anna,We highly recommend this accomodation and agre...
4,958,2010-02-13,V Jillian,Holly's place was great. It was exactly what I...


In [66]:
import numpy as np

# Assuming 'id', 'reviewer_id', and 'sentiment_score' are columns in your DataFrame 'review_total'
# and you want to convert these columns to integer type:

# Select the columns you want to convert
columns_to_convert = review_sdata[['id', 'reviewer_id', 'sentiment_score']]

# Convert selected columns to numeric (int if possible, otherwise float)
for column in columns_to_convert:
    try:
        review_sdata[column] = pd.to_numeric(review_sdata[column], errors='coerce').astype(int)
    except ValueError:
        print(f"Column '{column}' could not be converted to int. Converting to float instead.")
        review_sdata[column] = pd.to_numeric(review_sdata[column], errors='coerce').astype(float)

In [67]:
columns_to_convert.head()

,id,reviewer_id,sentiment_score
0,5977.0,15695.0,0.96
1,6660.0,26145.0,0.98
2,11519.0,25839.0,0.76
3,16282.0,33750.0,0.98
4,26008.0,15416.0,0.96


In [68]:
review_sdata.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments,sentiment_score
0,5883,5977,4,15695,11068,218351,0
1,5883,6660,5,26145,39116,237567,0
2,5883,11519,14,25839,9775,355658,0
3,5883,16282,18,33750,2626,345807,0
4,5883,26008,30,15416,43294,131685,0


In [69]:
review_sdata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 391343 entries, 0 to 1223501
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype
---  ------           --------------   -----
 0   listing_id       391343 non-null  int64
 1   id               391343 non-null  int64
 2   date             391343 non-null  int64
 3   reviewer_id      391343 non-null  int64
 4   reviewer_name    391343 non-null  int64
 5   comments         391343 non-null  int64
 6   sentiment_score  391343 non-null  int64
dtypes: int64(7)
memory usage: 23.9 MB


In [70]:
metadata=SingleTableMetadata()
metadata.detect_from_dataframe(review_sdata)

In [ ]:
import joblib
# ... other imports ...

# before calling synthesizer.fit:
joblib.parallel_backend('loky', n_jobs=2)
synthesizer=CTGANSynthesizer(metadata,epochs=500,verbose=True)
synthesizer.fit(review_sdata)

/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:120: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.10/dist-packages/sdv/single_table/base.py:105: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


In [ ]:
synthetic_data=synthesizer.sample(num_rows=1000)

In [ ]:
synthetic_data.shape

In [ ]:
synthetic_data.to_csv('review_Synthetic_data', index=False)